## <strong>VGG 16을 활용한 trash 분류 (Transfer Learning 활용)</strong>

- kaggle dataset(https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification)을 활용한 trash 분류 모델

## 순서

1. **import**: 필요한 모듈 import
2. **전처리**: 학습에 필요한 데이터 전처리를 수행합니다.
3. **모델링(model)**: 모델을 정의합니다.
4. **컴파일(compile)**: 모델을 생성합니다.
5. **학습 (fit)**: 모델을 학습시킵니다.

### 1) **import**: 필요한 모듈 import

In [18]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Flatten, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

### 2) **전처리**: 학습에 필요한 데이터 전처리를 수행합니다.

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

path='/content/gdrive/My Drive/AI/dataset/garbage-classification/Garbage-classification/Garbage-classification'

train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    rotation_range=30,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.1,
    horizontal_flip = True,
    fill_mode = 'nearest',
)

train_generator = train_datagen.flow_from_directory(
    directory=path, 
    target_size=(224,224), 
    classes=['cardboard', 'glass', 'metal','paper', 'plastic', 'trash'], 
    batch_size=16, 
    subset='training'
)

valid_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    validation_split=0.1,
    rescale=1./255,
)


validation_generator = valid_datagen.flow_from_directory(
    directory=path, 
    target_size=(224,224), 
    classes=['cardboard', 'glass', 'metal','paper', 'plastic', 'trash'], 
    batch_size=16, 
    subset='validation'
)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.


### 3) **모델링(model)**: 모델을 정의합니다.

In [4]:
transfer_model = VGG16(weights='imagenet', include_top = False, input_shape=(224,224,3))
transfer_model.trainable = False

model = Sequential([
  transfer_model,
  GlobalAveragePooling2D(),
  Dense(512, activation='relu'),
  BatchNormalization(),
  Dropout(0.3),
  Dense(128, activation='relu'),
  BatchNormalization(),
  Dropout(0.3),
  Dense(6, activation='softmax')
])

model.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                         

### 4) **컴파일(compile)**: 모델을 생성합니다.

In [5]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['acc'])

checkpoint_path = "garbage-classifiction-checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss', 
                             verbose=1)

### 5) **학습 (fit)**: 모델을 학습시킵니다.

In [9]:
model.fit(train_generator,
          validation_data=(validation_generator),
          epochs=100,
          callbacks=[checkpoint]
          )

model.load_weights(checkpoint_path)
model.save('garbage-classification-model1(epoch=30).h5')

Epoch 1/100
143/143 [==============================] - ETA: 0s - loss: 1.6648 - acc: 0.3888
Epoch 1: val_loss improved from inf to 1.47926, saving model to garbage-classifiction-checkpoint.ckpt
143/143 [==============================] - 655s 5s/step - loss: 1.6648 - acc: 0.3888 - val_loss: 1.4793 - val_acc: 0.4183
Epoch 2/100
143/143 [==============================] - ETA: 0s - loss: 1.4371 - acc: 0.4741
Epoch 2: val_loss improved from 1.47926 to 1.14780, saving model to garbage-classifiction-checkpoint.ckpt
143/143 [==============================] - 40s 278ms/step - loss: 1.4371 - acc: 0.4741 - val_loss: 1.1478 - val_acc: 0.5697
Epoch 3/100
143/143 [==============================] - ETA: 0s - loss: 1.2926 - acc: 0.5321
Epoch 3: val_loss improved from 1.14780 to 0.91768, saving model to garbage-classifiction-checkpoint.ckpt
143/143 [==============================] - 38s 266ms/step - loss: 1.2926 - acc: 0.5321 - val_loss: 0.9177 - val_acc: 0.6295
Epoch 4/100
143/143 [===================

### 모델2) 

In [25]:
transfer_model = VGG16(weights='imagenet', include_top = False, input_shape=(224,224,3))
transfer_model.trainable = False

model_2 = Sequential([
  transfer_model,
  Flatten(),
  Dropout(0.5),
  Dense(512, activation='relu'),
  Dense(128, activation='relu'),
  Dense(6, activation='softmax')
])

model_2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_4 (Flatten)         (None, 25088)             0         
                                                                 
 dropout_6 (Dropout)         (None, 25088)             0         
                                                                 
 dense_15 (Dense)            (None, 512)               12845568  
                                                                 
 dense_16 (Dense)            (None, 128)               65664     
                                                                 
 dense_17 (Dense)            (None, 6)                 774       
                                                                 
Total params: 27,626,694
Trainable params: 12,912,006


### 4) **모델2 컴파일(compile)**: 모델을 생성합니다.

In [26]:
model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['acc'])

checkpoint_path = "garbage-classifiction-checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss', 
                             verbose=1)

### 5) **모델2 학습 (fit)**: 모델을 학습시킵니다.

In [27]:
model_2.fit(train_generator,
          validation_data=(validation_generator),
          epochs=100,
          callbacks=[checkpoint]
          )

model_2.load_weights(checkpoint_path)
model_2.save('garbage-classification-model2(epoch=100).h5')

Epoch 1/100
143/143 [==============================] - ETA: 0s - loss: 1.3807 - acc: 0.4315
Epoch 1: val_loss improved from inf to 1.03882, saving model to garbage-classifiction-checkpoint.ckpt
143/143 [==============================] - 97s 676ms/step - loss: 1.3807 - acc: 0.4315 - val_loss: 1.0388 - val_acc: 0.5976
Epoch 2/100
143/143 [==============================] - ETA: 0s - loss: 1.0824 - acc: 0.5839
Epoch 2: val_loss improved from 1.03882 to 0.82796, saving model to garbage-classifiction-checkpoint.ckpt
143/143 [==============================] - 38s 268ms/step - loss: 1.0824 - acc: 0.5839 - val_loss: 0.8280 - val_acc: 0.6653
Epoch 3/100
143/143 [==============================] - ETA: 0s - loss: 0.9562 - acc: 0.6331
Epoch 3: val_loss improved from 0.82796 to 0.78093, saving model to garbage-classifiction-checkpoint.ckpt
143/143 [==============================] - 38s 267ms/step - loss: 0.9562 - acc: 0.6331 - val_loss: 0.7809 - val_acc: 0.7012
Epoch 4/100
143/143 [=================